# Restoring Early Stopping
> “The past is never where you think you left it.”
>
> &mdash; <cite>Katherine Anne Porter</cite>

In [ ]:
# default_exp restoring_early_stopping

In [ ]:
# hide
import blackhc.project.script
from nbdev.showdoc import *

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


In [ ]:
# exports

import pickle
import warnings
from math import inf
from typing import Callable, Optional, Dict

import torch
from ignite.engine import Engine, Events
from torch.optim import Optimizer
from torch.optim.lr_scheduler import EPOCH_DEPRECATION_WARNING


class RestoringEarlyStopping(object):
    """RestoringEarlyStopping handler can be used to stop the training if no improvement after a given number of events

    Args:
        patience (int):
            Number of events to wait if no improvement and then stop the training
        score_function (Callable):
            It should be a function taking a single argument, an `ignite.engine.Engine` object,
            and return a score `float`. An improvement is considered if the score is higher.
        trainer (Engine):
            trainer engine to stop the run if no improvement

    Examples:

    .. code-block:: python

        from ignite.engine import Engine, Events
        from ignite.handlers import EarlyStopping

        def score_function(engine):
            val_loss = engine.state.metrics['nll']
            return -val_loss

        handler = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
        # Note: the handler is attached to an *Evaluator* (runs one epoch on validation dataset)
        evaluator.add_event_handler(Events.COMPLETED, handler)

    """

    def __init__(
        self,
        *,
        patience,
        score_function,
        training_engine: Engine,
        validation_engine: Engine,
        module: torch.nn.Module = None,
        optimizer: torch.optim.Optimizer = None,
        out_of_patience_callback=None,
    ):
        if out_of_patience_callback is None:

            def default_out_of_patience_callback():
                training_engine.terminate()

            out_of_patience_callback = default_out_of_patience_callback

        if not callable(score_function):
            raise TypeError("Argument score_function should be a function")

        if patience < 0:
            raise ValueError("Argument patience should be non-negative integer")

        self.score_function = score_function
        self.out_of_patience_callback = out_of_patience_callback
        self.module = module
        self.optimizer = optimizer

        self.patience = patience
        self.counter = 0

        self.best_score = None
        self.best_epoch = None
        self.best_module_state_dict = None
        self.best_optimizer_state_dict = None
        self.restore_epoch = None

        self.training_engine = training_engine
        self.validation_engine = validation_engine
        validation_engine.add_event_handler(Events.EPOCH_COMPLETED, self.on_epoch_completed)
        training_engine.add_event_handler(Events.COMPLETED, self.on_completed)

    def snapshot(self):
        self.best_epoch = self.training_engine.state.epoch
        if self.module is not None:
            self.best_module_state_dict = pickle.dumps(self.module.state_dict(keep_vars=False))
        if self.optimizer is not None:
            self.best_optimizer_state_dict = pickle.dumps(self.optimizer.state_dict())

    def restore_best(self):
        if self.best_module_state_dict is not None and self.module is not None:
            print(f"RestoringEarlyStopping: Restoring best parameters. (Score: {self.best_score})")
            self.module.load_state_dict(pickle.loads(self.best_module_state_dict))

        if self.best_optimizer_state_dict is not None and self.optimizer is not None:
            print("RestoringEarlyStopping: Restoring optimizer.")
            self.optimizer.load_state_dict(pickle.loads(self.best_optimizer_state_dict))

    def on_epoch_completed(self, _):
        score = self.score_function()

        if self.best_score is not None and score <= self.best_score:
            self.counter += 1
            print("RestoringEarlyStopping: %i / %i" % (self.counter, self.patience))
            if self.counter >= self.patience:
                print("RestoringEarlyStopping: Out of patience")
                self.restore_best()
                self.restore_epoch = self.training_engine.state.epoch

                # Reset the counter in case we keep training after adjusting the model.
                self.counter = 0
                self.out_of_patience_callback()
        else:
            self.best_score = score
            self.snapshot()
            self.counter = 0

    def on_completed(self, _):
        if self.restore_epoch is None or self.restore_epoch < self.training_engine.state.epoch:
            self.restore_best()
            self.restore_epoch = self.training_engine.state.epoch

In [ ]:
# exports


class PatienceWithSnapshot(object):
    """PatienceSnapshot handler can be used to stop the training if no improvement after a given number of events

    Args:
        patience (int):
            Number of events to wait if no improvement and then stop the training
        score_function (Callable):
            It should be a function taking a single argument, an `ignite.engine.Engine` object,
            and return a score `float`. An improvement is considered if the score is higher.
        trainer (Engine):
            trainer engine to stop the run if no improvement

    Examples:

    .. code-block:: python

        from ignite.engine import Engine, Events
        from ignite.handlers import EarlyStopping

        def score_function(engine):
            val_loss = engine.state.metrics['nll']
            return -val_loss

        handler = EarlyStopping(patience=10, score_function=score_function, trainer=trainer)
        # Note: the handler is attached to an *Evaluator* (runs one epoch on validation dataset)
        evaluator.add_event_handler(Events.COMPLETED, handler)

    """

    def __init__(
        self,
        *,
        name: str = "",
        patience,
        score_function,
        training_engine: Engine,
        validation_engine: Engine,
        module: torch.nn.Module = None,
        optimizer: torch.optim.Optimizer = None,
        out_of_patience_callback=None,
    ):
        if out_of_patience_callback is None:

            def default_out_of_patience_callback(module_state_dict, optimizer_state_dict):
                training_engine.terminate()

            out_of_patience_callback = default_out_of_patience_callback

        if not callable(score_function):
            raise TypeError("Argument score_function should be a function")

        if patience < 0:
            raise ValueError("Argument patience should be non-negative integer")

        self.name = name
        self.score_function = score_function
        self.out_of_patience_callback = out_of_patience_callback
        self.module = module
        self.optimizer = optimizer

        self.patience = patience
        self.counter = 0

        self.best_score = None
        self.best_epoch = None
        self.best_module_state_dict = None
        self.best_optimizer_state_dict = None
        self.callback_epoch = None

        self.training_engine = training_engine
        self.validation_engine = validation_engine
        validation_engine.add_event_handler(Events.EPOCH_COMPLETED, self.on_epoch_completed)
        training_engine.add_event_handler(Events.COMPLETED, self.on_completed)

    def is_out_of_patience(self):
        return self.counter >= self.patience

    def snapshot(self):
        self.best_epoch = self.training_engine.state.epoch
        if self.module is not None:
            self.best_module_state_dict = pickle.dumps(self.module.state_dict(keep_vars=False))
        if self.optimizer is not None:
            self.best_optimizer_state_dict = pickle.dumps(self.optimizer.state_dict())

    def restore(self, module_state_dict, optimizer_state_dict):
        if self.best_module_state_dict is not None and self.module is not None:
            print(f"{self.name}PatienceWithSnapshot: Restoring best parameters. (Score: {self.best_score})")
            self.module.load_state_dict(pickle.loads(self.best_module_state_dict))

        if self.best_optimizer_state_dict is not None and self.optimizer is not None:
            print(f"{self.name}PatienceWithSnapshot: Restoring optimizer.")
            self.optimizer.load_state_dict(pickle.loads(self.best_optimizer_state_dict))

    def on_epoch_completed(self, _):
        score = self.score_function()

        if self.best_score is not None and score <= self.best_score:
            self.counter += 1
            print(f"{self.name}PatienceWithSnapshot: %i / %i" % (self.counter, self.patience))
            if self.counter == self.patience:
                print(f"{self.name}PatienceWithSnapshot: Out of patience")
                print(f"{self.name}PatienceWithSnapshot: Best score: {self.best_score})")
                self.callback_epoch = self.training_engine.state.epoch

                module_state_dict = pickle.loads(self.best_module_state_dict)
                optimizer_state_dict = pickle.loads(self.best_optimizer_state_dict)
                self.out_of_patience_callback(module_state_dict, optimizer_state_dict)
        else:
            self.best_score = score
            self.snapshot()
            self.counter = 0

    def on_completed(self, _):
        if self.callback_epoch is None or self.callback_epoch < self.training_engine.state.epoch:
            self.callback_epoch = self.training_engine.state.epoch

In [ ]:
# exports


def suggest_limit_schedule(patience_schedule, max_epochs, factor=2):
    backward_limit_schedule = []
    current_epoch = max_epochs
    for patience in reversed(patience_schedule):
        current_epoch -= factor * patience
        backward_limit_schedule += [current_epoch]

    limit_schedule = list(reversed(backward_limit_schedule))
    if limit_schedule[0] < 0:
        limit_schedule = []
        current_epoch = 0
        for patience in patience_schedule:
            current_epoch += factor * patience
            limit_schedule += [current_epoch]
        max_epochs = current_epoch

    return limit_schedule, max_epochs

In [ ]:
suggest_limit_schedule([20, 10], 120)

([60, 100], 120)

In [ ]:
suggest_limit_schedule([30, 20, 20], 120)

([60, 100, 140], 140)

In [ ]:
# exports

class ModuleOptimizerSnapshotter:
    module: torch.nn.Module
    optimizer: torch.optim.Optimizer
    module_state_dict: Optional
    optimizer_state_dict: Optional

    def __init__(self, module: torch.nn.Module, optimizer: torch.optim.Optimizer):
      self.module = module
      self.optimizer = optimizer
      self.module_state_dict = None
      self.optimizer_state_dict = None

    def snapshot(self):
        if self.module is not None:
            self.module_state_dict = pickle.dumps(self.module.state_dict(keep_vars=False))

            # Only take a snapshot of the optimizer if we also have a module.
            if self.optimizer is not None:
                self.optimizer_state_dict = pickle.dumps(self.optimizer.state_dict())

    def restore(self):
        if self.module_state_dict is not None and self.module is not None:
            self.module.load_state_dict(pickle.loads(self.module_state_dict))

            # Only restore if we also restore a module (otherwise... messy?)
            if self.optimizer_state_dict is not None and self.optimizer is not None:
                self.optimizer.load_state_dict(pickle.loads(self.optimizer_state_dict))


class ReduceLROnPlateauWithSchedule(object):
    """Reduce learning rate when a metric has stopped improving.
    Models often benefit from reducing the learning rate by a factor
    of 2-10 once learning stagnates. This scheduler reads a metrics
    quantity and if no improvement is seen for a 'patience' number
    of epochs, the learning rate is reduced.

    Args:
        optimizer (Optimizer): Wrapped optimizer.
        mode (str): One of `min`, `max`. In `min` mode, lr will
            be reduced when the quantity monitored has stopped
            decreasing; in `max` mode it will be reduced when the
            quantity monitored has stopped increasing. Default: 'min'.
        factor (float): Factor by which the learning rate will be
            reduced. new_lr = lr * factor. Default: 0.1.
        patience (int): Number of epochs with no improvement after
            which learning rate will be reduced. For example, if
            `patience = 2`, then we will ignore the first 2 epochs
            with no improvement, and will only decrease the LR after the
            3rd epoch if the loss still hasn't improved then.
            Default: 10.
        threshold (float): Threshold for measuring the new optimum,
            to only focus on significant changes. Default: 1e-4.
        threshold_mode (str): One of `rel`, `abs`. In `rel` mode,
            dynamic_threshold = best * ( 1 + threshold ) in 'max'
            mode or best * ( 1 - threshold ) in `min` mode.
            In `abs` mode, dynamic_threshold = best + threshold in
            `max` mode or best - threshold in `min` mode. Default: 'rel'.
        cooldown (int): Number of epochs to wait before resuming
            normal operation after lr has been reduced. Default: 0.
        min_lr (float or list): A scalar or a list of scalars. A
            lower bound on the learning rate of all param groups
            or each group respectively. Default: 0.
        eps (float): Minimal decay applied to lr. If the difference
            between new and old lr is smaller than eps, the update is
            ignored. Default: 1e-8.
        verbose (bool): If ``True``, prints a message to stdout for
            each update. Default: ``False``.

    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> scheduler = ReduceLROnPlateau(optimizer, 'min')
        >>> for epoch in range(10):
        >>>     train(...)
        >>>     val_loss = validate(...)
        >>>     # Note that step should be called after validate()
        >>>     scheduler.step(val_loss)
    """

    def __init__(
        self,
        optimizer: torch.optim.Optimizer,
        factor_schedule: [float],
        patience_schedule: [int],
        limit_schedule: [int],
        end_callback: Callable,
        next_era_callback: Callable,
        mode="min",
        threshold=1e-4,
        threshold_mode="rel",
        cooldown=0,
        min_lr=0,
        eps=1e-8,
        verbose=False,
        module: torch.nn.Module = None,
    ):
        if any(factor >= 1.0 for factor in factor_schedule):
            raise ValueError("Factor should be < 1.0.")
        self.factor_schedule = factor_schedule

        # Attach optimizer
        if not isinstance(optimizer, Optimizer):
            raise TypeError("{} is not an Optimizer".format(type(optimizer).__name__))
        self.optimizer = optimizer

        self.snapshotter = ModuleOptimizerSnapshotter(module=module, optimizer=optimizer)

        if isinstance(min_lr, list) or isinstance(min_lr, tuple):
            if len(min_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} min_lrs, got {}".format(len(optimizer.param_groups), len(min_lr)))
            self.min_lrs = list(min_lr)
        else:
            self.min_lrs = [min_lr] * len(optimizer.param_groups)

        self.era = 0
        self.patience = patience_schedule[0]
        self.factor = factor_schedule[0]
        self.limit = limit_schedule[0]

        self.end_callback = end_callback
        self.next_era_callback = next_era_callback

        self.patience_schedule = patience_schedule
        self.limit_schedule = limit_schedule
        self.verbose = verbose
        self.cooldown = cooldown
        self.cooldown_counter = 0
        self.mode = mode
        self.threshold = threshold
        self.threshold_mode = threshold_mode
        self.best = None
        self.best_epoch = None
        self.num_bad_epochs = None
        self.mode_worse = None  # the worse value for the chosen mode
        self.eps = eps
        self.last_epoch = 0
        self._init_is_better(mode=mode, threshold=threshold, threshold_mode=threshold_mode)
        self._reset()

    def _reset(self):
        """Resets num_bad_epochs counter and cooldown counter."""
        self.best = self.mode_worse
        self.best_epoch = None
        self.cooldown_counter = 0
        self.num_bad_epochs = 0

    def step(self, metrics, epoch=None):
        # convert `metrics` to float, in case it's a zero-dim Tensor
        current = float(metrics)
        if epoch is None:
            epoch = self.last_epoch + 1
        else:
            warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
        self.last_epoch = epoch

        if self.is_better(current, self.best):
            self.best = current
            self.best_epoch = epoch
            self.num_bad_epochs = 0
            self.snapshotter.snapshot()
        else:
            self.num_bad_epochs += 1
            if self.verbose:
                print(
                    f"Epoch {epoch}: {current} worse than {self.best}, patience: {self.num_bad_epochs}/{self.patience}!"
                )

        if self.in_cooldown:
            self.cooldown_counter -= 1
            self.num_bad_epochs = 0  # ignore any bad epochs in cooldown

        if self.num_bad_epochs > self.patience or self.last_epoch >= self.limit:
            if self.snapshotter.module:
              print(f"Restoring best snapshot: best_score: {self.best} @ epoch {self.best_epoch}")
              self.snapshotter.restore()

            self._reduce_lr(epoch)
            self.cooldown_counter = self.cooldown
            self.num_bad_epochs = 0

        self._last_lr = [group["lr"] for group in self.optimizer.param_groups]

    def _reduce_lr(self, epoch):
        self.era += 1
        if self.era >= len(self.patience_schedule):
            if self.end_callback:
                self.end_callback()
            return
        else:
            self.patience = self.patience_schedule[self.era]
            self.limit = self.limit_schedule[self.era]
            if self.next_era_callback:
                self.next_era_callback()

        for i, param_group in enumerate(self.optimizer.param_groups):
            old_lr = float(param_group["lr"])
            new_lr = max(old_lr * self.factor, self.min_lrs[i])
            if old_lr - new_lr > self.eps:
                param_group["lr"] = new_lr
                if self.verbose:
                    print("Epoch {:5d}: reducing learning rate" " of group {} to {:.4e}.".format(epoch, i, new_lr))

        if self.era < len(self.factor_schedule):
            self.factor = self.factor_schedule[self.era]

    @property
    def in_cooldown(self):
        return self.cooldown_counter > 0

    def is_better(self, a, best):
        if self.mode == "min" and self.threshold_mode == "rel":
            rel_epsilon = 1.0 - self.threshold
            return a < best * rel_epsilon

        elif self.mode == "min" and self.threshold_mode == "abs":
            return a < best - self.threshold

        elif self.mode == "max" and self.threshold_mode == "rel":
            rel_epsilon = self.threshold + 1.0
            return a > best * rel_epsilon

        else:  # mode == 'max' and epsilon_mode == 'abs':
            return a > best + self.threshold

    def _init_is_better(self, mode, threshold, threshold_mode):
        if mode not in {"min", "max"}:
            raise ValueError("mode " + mode + " is unknown!")
        if threshold_mode not in {"rel", "abs"}:
            raise ValueError("threshold mode " + threshold_mode + " is unknown!")

        if mode == "min":
            self.mode_worse = inf
        else:  # mode == 'max':
            self.mode_worse = -inf

        self.mode = mode
        self.threshold = threshold
        self.threshold_mode = threshold_mode

    def state_dict(self):
        return {key: value for key, value in self.__dict__.items() if key != "optimizer"}

    def load_state_dict(self, state_dict):
        self.__dict__.update(state_dict)
        self._init_is_better(mode=self.mode, threshold=self.threshold, threshold_mode=self.threshold_mode)


def lr_step_after_epoch(trainer, scheduler):
    if isinstance(scheduler, ReduceLROnPlateauWithScheduleWrapper):

        @trainer.on(Events.EPOCH_COMPLETED)
        def lr_step(engine):
            scheduler.step(engine)

    elif isinstance(scheduler, ReduceLROnPlateauWithSchedule):
        warnings.warn(
            "Use ReduceLROnPlateauWrapper instead of torch.optim.lr_scheduler.ReduceLROnPlateau!", DeprecationWarning
        )

        @trainer.on(Events.EPOCH_COMPLETED)
        def lr_step(engine):
            scheduler.step(engine.state.output.loss)

    elif isinstance(scheduler, torch.optim.lr_scheduler.CosineAnnealingWarmRestarts):

        @trainer.on(Events.GET_BATCH_COMPLETED)
        def lr_step(engine):
            scheduler.step(engine.state.epoch + engine.state.iteration / engine.state.epoch_length)

    else:

        @trainer.on(Events.EPOCH_COMPLETED)
        def lr_step(engine):
            scheduler.step()


class ReduceLROnPlateauWithScheduleWrapper(ReduceLROnPlateauWithSchedule):
    output_transform: Optional[Callable]
    metrics_transform: Optional[Callable]

    def __init__(
        self,
        optimizer,
        *,
        output_transform: Optional[Callable] = None,
        metrics_transform: Optional[Callable] = None,
        factor_schedule: [float],
        patience_schedule: [int],
        limit_schedule: [int],
        next_era_callback: Callable,
        end_callback: Callable,
        mode="min",
        threshold=1e-4,
        threshold_mode="rel",
        cooldown=0,
        min_lr=0,
        eps=1e-8,
        verbose=False,
        module: torch.nn.Module = None,
    ):
        super().__init__(
            optimizer,
            factor_schedule=factor_schedule,
            patience_schedule=patience_schedule,
            limit_schedule=limit_schedule,
            end_callback=end_callback,
            next_era_callback=next_era_callback,
            mode=mode,
            threshold=threshold,
            threshold_mode=threshold_mode,
            cooldown=cooldown,
            min_lr=min_lr,
            eps=eps,
            verbose=verbose,
            module=module
        )
        self.metrics_transform = metrics_transform
        self.output_transform = output_transform
        assert self.metrics_transform or self.output_transform
        assert not self.metrics_transform or not self.output_transform

    # noinspection PyMethodOverriding
    def step(self, engine: Engine):
        if self.output_transform:
            super().step(self.output_transform(engine.state.output))
        elif self.metrics_transform:
            super().step(self.metrics_transform(engine.state.metrics))
        else:
            raise NotImplementedError()